In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import models, transforms, datasets
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
import time
import os
import copy
from pathlib import Path
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#a=[]
#while(1):
#  a.append('1')

In [3]:
!git clone https://github.com/sheharyar19/Wheat-Data.git

Cloning into 'Wheat-Data'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 760 (delta 0), reused 4 (delta 0), pack-reused 750
Receiving objects: 100% (760/760), 1.30 GiB | 14.46 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (877/877), done.


In [4]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(hue=.05, saturation=.05),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
train_set = dsets.ImageFolder(root="Wheat-Data/train", transform=transform_train)
print("Classes: ")
class_names=train_set.classes
print(class_names)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
print("Total Training Images:",len(train_loader))
print(train_set.class_to_idx)

Classes: 
['healthy_wheat', 'leaf_rust', 'stem_rust']
Total Training Images: 11
{'healthy_wheat': 0, 'leaf_rust': 1, 'stem_rust': 2}


In [5]:
dataiter = iter(train_loader)
img, lbl = dataiter.next()
print(img.shape, lbl.shape)

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [6]:
transform_test = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor()])  
test_set = dsets.ImageFolder("Wheat-Data/test", transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)
print("Total Test Images:",len(test_set))

Total Test Images: 214


In [ ]:
!git clone https://github.com/sheharyar19/Wheat-Data-2.git

fatal: destination path 'Wheat-Data-2' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/sheharyar19/Wheat-Test.git

fatal: destination path 'Wheat-Test' already exists and is not an empty directory.


In [ ]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
train_set = dsets.ImageFolder(root="Wheat-Data-2/train", transform=transform_train)
print("Classes: ")
class_names=train_set.classes
print(class_names)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
print("Total Training Images:",len(train_loader))
print(train_set.class_to_idx)

Classes: 
['healthy_wheat', 'leaf_rust', 'stem_rust']
Total Training Images: 14
{'healthy_wheat': 0, 'leaf_rust': 1, 'stem_rust': 2}


In [ ]:
class ImageFolderWithPaths (datasets.ImageFolder):
      def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
transform_test = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor()])  
test_set = ImageFolderWithPaths("Wheat-Test/test", transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)
print("Total Test Images:",len(test_set))

Total Test Images: 610


In [ ]:
vgg = torchvision.models.vgg16(pretrained=True)
num_features = vgg.classifier[6].in_features
features = list(vgg.classifier.children())[:-1] # Remove last layer
features.extend(nn.Sequential(nn.Linear(num_features, 1024), nn.ReLU(inplace=True), nn.Linear(1024, 3)))
vgg.classifier = nn.Sequential(*features)
vgg.classifier = nn.Sequential(*list(vgg.classifier) + [nn.Softmax(1)])

vgg = vgg.to(device)
print(vgg.classifier)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth



Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1024, bias=True)
  (7): ReLU(inplace=True)
  (8): Linear(in_features=1024, out_features=3, bias=True)
  (9): Softmax(dim=1)
)


In [7]:
vgg = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [8]:
print(vgg)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
vgg.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 3)
)

In [ ]:
vgg.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2208, 3)
)

In [10]:
vgg.fc =  nn.Sequential(*list(vgg.fc) + [nn.Softmax(1)])
#vgg.classifier = nn.Sequential(*list(vgg.classifier) + [nn.Softmax(1)])


vgg = vgg.to(device)

In [ ]:
vgg.classifier =  nn.Sequential(*list(vgg.classifier) + [nn.Softmax(1)])
#vgg.classifier = nn.Sequential(*list(vgg.classifier) + [nn.Softmax(1)])


vgg = vgg.to(device)

In [ ]:
print(vgg)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [11]:
vgg = vgg.to(device)

In [12]:
Epochs = 35
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.parameters(), lr=0.0001)

In [13]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device

vgg.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(train_loader.dataset),
                100. * i / len(train_loader),
                loss.item()))
                
# print('Starting')
# print('Running for',Epochs,'Epochs')
# print('Batch Size:', train_loader.batch_size)
# print('No of batches per epoch:', int(len(train_set)/train_loader.batch_size))
# total_loss = []
# for epoch in range(Epochs):
#     listLoss = []
#     for i,data in enumerate(train_loader,0):
#             inputs, labels = data
#             inputs, labels = inputs.to(device), labels.to(device)
#             print(inputs.shape, labels.shape)
#             optimizer.zero_grad()
#             outputs = vgg(inputs)               #----> forward pass
#             loss = criterion(outputs, labels)   #----> compute loss
#             loss.backward()                     #----> backward pass
#             optimizer.step()                    #----> weights update
#             listLoss.append(loss.item())
#             if (((i+1) % 10) == 0):
#                 print("\rEpoch #:",epoch+1, "  Batch #:", i+1,end="")
#     l = np.sum(listLoss)/(len(listLoss))
#     print('\nEpoch #:', epoch+1, '  Training Loss:', l)
#     total_loss.append(l)
# print('Finished Training')

Train Epoch: 0 [210/661 (91%)]	Loss: 0.866970: : 11it [01:19,  7.23s/it]
Train Epoch: 1 [210/661 (91%)]	Loss: 0.851500: : 11it [01:19,  7.19s/it]
Train Epoch: 2 [210/661 (91%)]	Loss: 0.700437: : 11it [01:18,  7.14s/it]
Train Epoch: 3 [210/661 (91%)]	Loss: 0.585319: : 11it [01:18,  7.15s/it]
Train Epoch: 4 [210/661 (91%)]	Loss: 0.659316: : 11it [01:18,  7.10s/it]
Train Epoch: 5 [210/661 (91%)]	Loss: 0.555624: : 11it [01:17,  7.05s/it]
Train Epoch: 6 [210/661 (91%)]	Loss: 0.565815: : 11it [01:17,  7.04s/it]
Train Epoch: 7 [210/661 (91%)]	Loss: 0.555713: : 11it [01:17,  7.06s/it]
Train Epoch: 8 [210/661 (91%)]	Loss: 0.592878: : 11it [01:16,  6.99s/it]
Train Epoch: 9 [210/661 (91%)]	Loss: 0.650436: : 11it [01:17,  7.08s/it]
Train Epoch: 10 [210/661 (91%)]	Loss: 0.587821: : 11it [01:17,  7.06s/it]
Train Epoch: 11 [210/661 (91%)]	Loss: 0.559006: : 11it [01:17,  7.08s/it]
Train Epoch: 12 [210/661 (91%)]	Loss: 0.555518: : 11it [01:17,  7.08s/it]
Train Epoch: 13 [210/661 (91%)]	Loss: 0.566863: 

In [1]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

NameError: ignored

In [ ]:
correct=0
total=0
pred=[]
names=[]
with torch.no_grad():
    for data in test_loader:
        images, labels, paths  = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.extend(outputs.data.tolist())
        names.extend(paths)
        total+=labels.size(0)
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 0 %


In [ ]:
img_names=[]
for i in names:
  img_names.append(Path(i).stem)


In [ ]:
df=pd.DataFrame(pred,columns=['healthy_wheat','leaf_rust', 'stem_rust'])

In [ ]:
df['ID']=img_names

In [ ]:
df=df[['ID','leaf_rust','stem_rust','healthy_wheat']]

In [ ]:
from google.colab import files
df.to_csv('test_results_resnetadam.csv', index=False) 
files.download('test_results_resnetadam.csv')

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
    model_save_name = 'densenet_weights_ovrfit.pt'
    path = F"/content/gdrive/My Drive/{model_save_name}" 
    torch.save(vgg.state_dict(), path)

In [ ]:
model_save_name = 'densenet_weights_ovr.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
vgg.load_state_dict(torch.load(path))
vgg.eval()


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 